In [14]:
# Import libraries
import pickle
import pandas as pd
import numpy as np
import sklearn

In [9]:
# Load in the library
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/albwrekt/.local/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.2.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/albwrekt/.local/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [10]:
# Read in the taxi data and generate the unique ID's
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [20]:
# Read the file
file_year = 2022
file_month = 2
df = read_data(f'./data/yellow_tripdata_{file_year:04d}-{file_month:02d}.parquet')

In [21]:
# Transform the pandas dataframe to dictionary
dicts = df[categorical].to_dict(orient='records')
# Use dictionary vectorizer to transform the output
X_val = dv.transform(dicts)
# Predict with the newly generated outputs, to the saved model.
y_pred = model.predict(X_val)

In [22]:
# Calculate the standard deviation of the duration predictions
print(f"Standard Deviation of Predictions: {np.std(y_pred)}")

Standard Deviation of Predictions: 5.28140357655334


In [25]:
# Create the ride ID, needs to be done this way to get the vectorized application of pandas
df['ride_id'] = f'{file_year:04d}/{file_month:02d}_'+df.index.astype('str')

In [28]:
# Write the ride_id and predictions to a results dataframe
df_results = df.copy()[['ride_id']]
df_results['duration'] = y_pred
df_results.head()

,ride_id,duration
0,2022/02_0,18.527783
1,2022/02_1,23.065782
2,2022/02_2,33.686359
3,2022/02_3,23.757436
4,2022/02_4,21.492904


In [30]:
# Save the data back out to parquet
df_results.to_parquet(
    path='output_file.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)